In [20]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import pandas as pd
from getpass import getpass

# Set the display options
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Connect to Neo4j
uri = "bolt://localhost:11005"  #change as needed
user = "neo4j"
password = getpass()

gds = GraphDataScience(uri, auth=(user,password))
gds.set_database("recipe")  #change as needed

 ········


# Create Projection

In [32]:
gds.run_cypher("""

MATCH (source:Recipe)-[:USES]->(target:Ingredient)
WHERE source.Recipe_Title CONTAINS 'cornbread'
WITH gds.graph.project(
  'ingredients',
  source,
  target,
  {},
  {undirectedRelationshipTypes: ['*']}
) as g
RETURN
  g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels

""")

,graph,nodes,rels
0,ingredients,8662,105426


### PageRank

In [33]:
gds.run_cypher("""

CALL gds.pageRank.write('ingredients', { writeProperty: 'pageRank' })

""")

,writeMillis,nodePropertiesWritten,ranIterations,didConverge,centralityDistribution,postProcessingMillis,preProcessingMillis,computeMillis,configuration
0,274,8662,20,False,"{'p99': 6.081786155700684, 'min': 0.21033954620361328, 'max': 231.11230373382568, 'mean': 0.9612402779432291, 'p90': 0.9673337936401367, 'p50': 0.6013288497924805, 'p999': 71.80761623382568, 'p95': 1.2656011581420898, 'p75': 0.7592916488647461}",126,0,149,"{'maxIterations': 20, 'writeConcurrency': 4, 'concurrency': 4, 'jobId': '991e5be8-28e0-4669-8b2e-7d71f8ad0183', 'sourceNodes': [], 'writeProperty': 'pageRank', 'scaler': 'NONE', 'logProgress': True, 'nodeLabels': ['*'], 'sudo': False, 'dampingFactor': 0.85, 'relationshipTypes': ['*'], 'tolerance': 1e-07}"


In [30]:
gds.run_cypher("""

MATCH (r:Recipe)-[:USES]->(i:Ingredient)
WHERE r.Recipe_Title CONTAINS 'cornbread'
WITH DISTINCT i
RETURN i.Ingredient_Name AS Ingredient, i.pageRank AS PageRank
ORDER BY i.pageRank DESC
LIMIT 10

""")

,Ingredient,PageRank
0,eggs,54.182461
1,salt,46.786139
2,onion,38.607501
3,milk,28.057664
4,sugar,24.776241
5,baking powder,24.219077
6,flour,20.145594
7,egg,18.830970
8,buttermilk,16.750039
9,butter,15.792249


# Drop Projection

In [31]:
gds.run_cypher("""

CALL gds.graph.drop('ingredients')

""")

,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
0,ingredients,recipe,,-1,8662,52713,"{'jobId': 'ec38982f-5cdb-49fd-9940-212b6a41e70e', 'creationTime': 2023-12-04T08:03:33.958933600-05:00, 'inverseIndexedRelationshipTypes': [], 'logProgress': True, 'undirectedRelationshipTypes': []}",0.000703,2023-12-04T08:03:33.958933600-05:00,2023-12-04T08:03:35.094594200-05:00,"{'graphProperties': {}, 'relationships': {'__ALL__': {}}, 'nodes': {'__ALL__': {}}}","{'graphProperties': {}, 'relationships': {'__ALL__': {'properties': {}, 'direction': 'DIRECTED'}}, 'nodes': {'__ALL__': {}}}"
